In [4]:
import re
import pandas as pd 
from urllib.parse import urlparse, parse_qs, urlunparse, urlencode
import random
import ast
import os 

In [41]:

links = pd.read_csv('/Users/alisa/Desktop/PittNAIL/cleardashboard_reviews/webscraping/best_buy_links.csv')
links.head()




,Unique ID,Product Name,Model,Company,search,best_buy_link
0,1,Jabra Enhance,Enhance Select 200 Premium,GN Resound,Jabra EnhanceEnhance Select 200 Premium,NaN
1,2,Jabra Enhance,Enhance Select 200 Basic,GN Resound,Jabra EnhanceEnhance Select 200 Basic,NaN
2,3,Jabra Enhance,Enhance Select 100 Premium,GN Resound,Jabra EnhanceEnhance Select 100 Premium,NaN
3,4,Jabra Enhance,Enhance Select 100 Basic,GN Resound,Jabra EnhanceEnhance Select 100 Basic,NaN
4,5,Jabra Enhance,Enhance Select 50 Premium,GN Resound,Jabra EnhanceEnhance Select 50 Premium,https://www.bestbuy.com/site/jabra-enhance-sel...


In [42]:

available = links[links['best_buy_link'].notna()]
available.head()
# for index, row in links:
#     if row.loc['amazon_link'] 


,Unique ID,Product Name,Model,Company,search,best_buy_link
4,5,Jabra Enhance,Enhance Select 50 Premium,GN Resound,Jabra EnhanceEnhance Select 50 Premium,https://www.bestbuy.com/site/jabra-enhance-sel...
6,182,Jabra Enhance,Enhance Select 500 Premium,GN Resound,Jabra EnhanceEnhance Select 500 Premium,https://www.bestbuy.com/site/jabra-enhance-sel...
8,184,Jabra Enhance,Enhance Select 300 Premium,GN Resound,Jabra EnhanceEnhance Select 300 Premium,https://www.bestbuy.com/site/jabra-enhance-sel...
12,9,Eargo,Eargo 7,Eargo 7,EargoEargo 7,https://www.bestbuy.com/site/eargo-7-self-fitt...
13,10,Lexie,Lexie Lumen,hearX Group,LexieLexie Lumen,https://www.bestbuy.com/site/lexie-hearing-lex...


### example of what links look like
 
1. main link (original)
https://www.bestbuy.com/site/jabra-enhance-select-50r-rechargeable-hearing-aids-with-remote-professional-care-and-bluetooth-streaming-gray/6558670.p?skuId=6558670



2. first page of all reviews 
https://www.bestbuy.com/site/reviews/jabra-enhance-select-50r-rechargeable-hearing-aids-with-remote-professional-care-and-bluetooth-streaming-gray/6558670?variant=A

3. second page 
https://www.bestbuy.com/site/reviews/jabra-enhance-select-50r-rechargeable-hearing-aids-with-remote-professional-care-and-bluetooth-streaming-gray/6558670?variant=A&page=2

4. third page 
https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/product-reviews/B09NWCSS86/ref=cm_cr_getr_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews&pageNumber=3 


In [14]:
def important_info(original_link):
    parsed_url = urlparse(original_link)

    path_parts = parsed_url.path.split('/') 
    # print(path_parts)
    if len(path_parts) >= 4:
        product_name = path_parts[2]
        product_id = path_parts[3]
        pattern = r'^\d+'

        match = re.match(pattern, product_id)

        if match:
            numeric_part = match.group()
            product_id = numeric_part

        # print(main_product_link)
        # https://www.amazon.com/EARGO-Hearing-Aids-Adults-Rechargeable/dp/B09NWCSS86/
        return product_name, product_id
    return "empty"
    


In [43]:
available = available.copy()

# shorten_amazon_url(available['amazon_link'].iloc[6])

In [44]:
available.loc[:, 'product_name'] = available['best_buy_link'].apply(lambda x: important_info(x)[0])
available.loc[:, 'product_id'] = available['best_buy_link'].apply(lambda x: important_info(x)[1])


available.head()
# available['Unique ID'].iloc[6]

,Unique ID,Product Name,Model,Company,search,best_buy_link,product_name,product_id
4,5,Jabra Enhance,Enhance Select 50 Premium,GN Resound,Jabra EnhanceEnhance Select 50 Premium,https://www.bestbuy.com/site/jabra-enhance-sel...,jabra-enhance-select-50r-rechargeable-hearing-...,6558670
6,182,Jabra Enhance,Enhance Select 500 Premium,GN Resound,Jabra EnhanceEnhance Select 500 Premium,https://www.bestbuy.com/site/jabra-enhance-sel...,jabra-enhance-select-500-gold,6584784
8,184,Jabra Enhance,Enhance Select 300 Premium,GN Resound,Jabra EnhanceEnhance Select 300 Premium,https://www.bestbuy.com/site/jabra-enhance-sel...,jabra-enhance-select-300-sparkling-silver,6589072
12,9,Eargo,Eargo 7,Eargo 7,EargoEargo 7,https://www.bestbuy.com/site/eargo-7-self-fitt...,eargo-7-self-fitting-otc-cic-hearing-aid-black,6532925
13,10,Lexie,Lexie Lumen,hearX Group,LexieLexie Lumen,https://www.bestbuy.com/site/lexie-hearing-lex...,lexie-hearing-lexie-lumen-self-fitting-otc-hea...,6482035


In [45]:
available = available.copy()



2. first page of all reviews 
https://www.bestbuy.com/site/reviews/jabra-enhance-select-50r-rechargeable-hearing-aids-with-remote-professional-care-and-bluetooth-streaming-gray/6558670?variant=A

In [46]:
def generate_first_page_reviews_link(link):
    parsed_url = urlparse(link)
    product_name = important_info(link)[0]
    product_id = important_info(link)[1]
    
    review_link = f"{parsed_url.scheme}://{parsed_url.netloc}/site/reviews/{product_name}/{product_id}?variant=A"
    return review_link




In [47]:
available.loc[:, 'review1'] = available['best_buy_link'].apply(generate_first_page_reviews_link)



2. first page of all reviews 
https://www.bestbuy.com/site/reviews/jabra-enhance-select-50r-rechargeable-hearing-aids-with-remote-professional-care-and-bluetooth-streaming-gray/6558670?variant=A

3. second page 
https://www.bestbuy.com/site/reviews/jabra-enhance-select-50r-rechargeable-hearing-aids-with-remote-professional-care-and-bluetooth-streaming-gray/6558670?variant=A&page=2



In [48]:
# list of review pages 2-10

def generate_reviews2_10(link):
    review_urls_list = []
    parsed_url = urlparse(link)

    # print(path_parts)
    product_name = important_info(link)[0]
    product_id = important_info(link)[1]
    for i in range(2,11):
        review_link = f"{parsed_url.scheme}://{parsed_url.netloc}/site/reviews/{product_name}/{product_id}?variant=A&page={i}"
        review_urls_list.append(review_link)
    # review_urls_list = review_urls_list.tolist()
    # print(type(review_urls_list))
    return review_urls_list

In [55]:
available.loc[:, 'reviews2-10'] = available['best_buy_link'].apply(generate_reviews2_10)
available = available.drop(16, 105)


available.head()


,Unique ID,Product Name,Model,Company,search,best_buy_link,product_name,product_id,review1,reviews2-10
4,5,Jabra Enhance,Enhance Select 50 Premium,GN Resound,Jabra EnhanceEnhance Select 50 Premium,https://www.bestbuy.com/site/jabra-enhance-sel...,jabra-enhance-select-50r-rechargeable-hearing-...,6558670,https://www.bestbuy.com/site/reviews/jabra-enh...,[https://www.bestbuy.com/site/reviews/jabra-en...
6,182,Jabra Enhance,Enhance Select 500 Premium,GN Resound,Jabra EnhanceEnhance Select 500 Premium,https://www.bestbuy.com/site/jabra-enhance-sel...,jabra-enhance-select-500-gold,6584784,https://www.bestbuy.com/site/reviews/jabra-enh...,[https://www.bestbuy.com/site/reviews/jabra-en...
8,184,Jabra Enhance,Enhance Select 300 Premium,GN Resound,Jabra EnhanceEnhance Select 300 Premium,https://www.bestbuy.com/site/jabra-enhance-sel...,jabra-enhance-select-300-sparkling-silver,6589072,https://www.bestbuy.com/site/reviews/jabra-enh...,[https://www.bestbuy.com/site/reviews/jabra-en...
12,9,Eargo,Eargo 7,Eargo 7,EargoEargo 7,https://www.bestbuy.com/site/eargo-7-self-fitt...,eargo-7-self-fitting-otc-cic-hearing-aid-black,6532925,https://www.bestbuy.com/site/reviews/eargo-7-s...,[https://www.bestbuy.com/site/reviews/eargo-7-...
13,10,Lexie,Lexie Lumen,hearX Group,LexieLexie Lumen,https://www.bestbuy.com/site/lexie-hearing-lex...,lexie-hearing-lexie-lumen-self-fitting-otc-hea...,6482035,https://www.bestbuy.com/site/reviews/lexie-hea...,[https://www.bestbuy.com/site/reviews/lexie-he...


In [56]:
available.head()


,Unique ID,Product Name,Model,Company,search,best_buy_link,product_name,product_id,review1,reviews2-10
4,5,Jabra Enhance,Enhance Select 50 Premium,GN Resound,Jabra EnhanceEnhance Select 50 Premium,https://www.bestbuy.com/site/jabra-enhance-sel...,jabra-enhance-select-50r-rechargeable-hearing-...,6558670,https://www.bestbuy.com/site/reviews/jabra-enh...,[https://www.bestbuy.com/site/reviews/jabra-en...
6,182,Jabra Enhance,Enhance Select 500 Premium,GN Resound,Jabra EnhanceEnhance Select 500 Premium,https://www.bestbuy.com/site/jabra-enhance-sel...,jabra-enhance-select-500-gold,6584784,https://www.bestbuy.com/site/reviews/jabra-enh...,[https://www.bestbuy.com/site/reviews/jabra-en...
8,184,Jabra Enhance,Enhance Select 300 Premium,GN Resound,Jabra EnhanceEnhance Select 300 Premium,https://www.bestbuy.com/site/jabra-enhance-sel...,jabra-enhance-select-300-sparkling-silver,6589072,https://www.bestbuy.com/site/reviews/jabra-enh...,[https://www.bestbuy.com/site/reviews/jabra-en...
12,9,Eargo,Eargo 7,Eargo 7,EargoEargo 7,https://www.bestbuy.com/site/eargo-7-self-fitt...,eargo-7-self-fitting-otc-cic-hearing-aid-black,6532925,https://www.bestbuy.com/site/reviews/eargo-7-s...,[https://www.bestbuy.com/site/reviews/eargo-7-...
13,10,Lexie,Lexie Lumen,hearX Group,LexieLexie Lumen,https://www.bestbuy.com/site/lexie-hearing-lex...,lexie-hearing-lexie-lumen-self-fitting-otc-hea...,6482035,https://www.bestbuy.com/site/reviews/lexie-hea...,[https://www.bestbuy.com/site/reviews/lexie-he...


In [155]:

# output_file_path = 'amazon_urls.csv'
# available.to_csv(output_file_path, index=False) 




## Crawl 

In [ ]:
# links is the same as available df 
# links = pd.read_csv('/Users/alisa/Desktop/PittNAIL/cleardashboard_reviews/amazon_urls.csv')
# links.head()

In [52]:
from selenium import webdriver
import time

In [161]:
# init_links = links['short_url'].copy()
# first_revpg = links['review1'].copy()
# reviews2_10 = links['reviews2-10'].copy()


In [59]:
def crawl(driver, link, filename, directory):
    full_path = os.path.join(directory, f'{filename}.html')
    driver.get('https://www.bestbuy.com')
    time.sleep(random.randint(6, 22))
    driver.get(link) 
    time.sleep(random.randint(10, 15))
    html_source=driver.page_source
    with open(full_path, 'w', encoding='utf-8') as file:
        file.write(html_source)
    time.sleep(random.randint(10, 16))

    

In [69]:
i=0

driver = webdriver.Chrome()
driver.get('https://www.bestbuy.com')
time.sleep(20)

for index, row in available.iterrows():
    row_uid = row.loc['Unique ID']
    if index== 6:
        first_revpg = row.loc['review1']
        reviews2_10 = row.loc['reviews2-10']
        parent_directory = f'./bestbuy pages'

        directory = os.path.join(parent_directory, str(row_uid))
        if not os.path.isdir(directory):
            os.makedirs(directory, exist_ok=True)
            full_path = os.path.join(directory, f'{row_uid}_revpg1.html')
            crawl(driver, first_revpg, f'{row_uid}_revpg1', directory)
            print(f"successfully downloaded page: {row_uid}_revpg1")
            for index, revpage in enumerate(reviews2_10):
                index +=2
                full_path = os.path.join(directory, f'{row_uid}_revpg{index}.html')
                crawl(driver, revpage, f'{row_uid}_revpg{index}', directory)
                print(f"successfully downloaded page: {row_uid}_revpg{index}")
                # get all the other review pages
            
        
        i+=1
        

successfully downloaded page: 182_revpg1
successfully downloaded page: 182_revpg2
successfully downloaded page: 182_revpg3
successfully downloaded page: 182_revpg4
successfully downloaded page: 182_revpg5
successfully downloaded page: 182_revpg6


KeyboardInterrupt: 